<a href="https://colab.research.google.com/github/douglasmmachado/MedicineConsumption/blob/master/notebooks/division_approach/6_Forecasting_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 - Forecasting and prediction validation



---



---



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math as m

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,  mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle


df_h1_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/agglomerative_clustering/datasets/division_approach/clustered/df_h1_clustered.csv"
df_h2_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/agglomerative_clustering/datasets/division_approach/clustered/df_h2_clustered.csv"
df_h3_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/agglomerative_clustering/datasets/division_approach/clustered/df_h3_clustered.csv"
df_h4_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/agglomerative_clustering/datasets/division_approach/clustered/df_h4_clustered.csv"



df_h1 = pd.read_csv(df_h1_url)
df_h1['YEAR'] = df_h1['YEAR'].astype(int)
df_h1['MONTH'] = df_h1['MONTH'].astype(int)
df_h1['DATE'] = pd.to_datetime(df_h1['YEAR'].astype(str) + '-' + df_h1['MONTH'].astype(str), format='%Y-%m')

df_h2 = pd.read_csv(df_h2_url)
df_h2['YEAR'] = df_h2['YEAR'].astype(int)
df_h2['MONTH'] = df_h2['MONTH'].astype(int)
df_h2['DATE'] = pd.to_datetime(df_h2['YEAR'].astype(str) + '-' + df_h2['MONTH'].astype(str), format='%Y-%m')

df_h3 = pd.read_csv(df_h3_url)
df_h3['YEAR'] = df_h3['YEAR'].astype(int)
df_h3['MONTH'] = df_h3['MONTH'].astype(int)
df_h3['DATE'] = pd.to_datetime(df_h3['YEAR'].astype(str) + '-' + df_h3['MONTH'].astype(str), format='%Y-%m')

df_h4 = pd.read_csv(df_h4_url)
df_h4['YEAR'] = df_h4['YEAR'].astype(int)
df_h4['MONTH'] = df_h4['MONTH'].astype(int)
df_h4['DATE'] = pd.to_datetime(df_h4['YEAR'].astype(str) + '-' + df_h4['MONTH'].astype(str), format='%Y-%m')

In [ ]:
df_h1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CODE_ATC        525 non-null    int64         
 1   DATE            525 non-null    datetime64[ns]
 2   HOSPI_CODE_UCD  525 non-null    int64         
 3   LIT_HC          525 non-null    float64       
 4   LIT_HP          525 non-null    float64       
 5   MONTH           525 non-null    int64         
 6   N_UFS           525 non-null    float64       
 7   PN_MEDICAL      525 non-null    float64       
 8   POPULATION      525 non-null    float64       
 9   P_MEDICAL       525 non-null    float64       
 10  QUANTITY        525 non-null    float64       
 11  QUANTITY_MA     525 non-null    float64       
 12  SEJ_MCO         525 non-null    float64       
 13  SEJ_SLD         525 non-null    float64       
 14  SEJ_SSR         525 non-null    float64       
 15  WEEK  

In [ ]:
df_h2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CODE_ATC        525 non-null    int64         
 1   DATE            525 non-null    datetime64[ns]
 2   HOSPI_CODE_UCD  525 non-null    int64         
 3   LIT_HC          525 non-null    float64       
 4   LIT_HP          525 non-null    float64       
 5   MONTH           525 non-null    int64         
 6   N_UFS           525 non-null    float64       
 7   PN_MEDICAL      525 non-null    float64       
 8   POPULATION      525 non-null    float64       
 9   P_MEDICAL       525 non-null    float64       
 10  QUANTITY        525 non-null    float64       
 11  QUANTITY_MA     525 non-null    float64       
 12  SEJ_MCO         525 non-null    float64       
 13  SEJ_PSY         525 non-null    float64       
 14  SEJ_SSR         525 non-null    float64       
 15  WEEK  

In [ ]:
df_h3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CODE_ATC        525 non-null    int64         
 1   DATE            525 non-null    datetime64[ns]
 2   HOSPI_CODE_UCD  525 non-null    int64         
 3   LIT_HC          525 non-null    float64       
 4   LIT_HP          525 non-null    float64       
 5   MONTH           525 non-null    int64         
 6   N_UFS           525 non-null    float64       
 7   PN_MEDICAL      525 non-null    float64       
 8   POPULATION      525 non-null    float64       
 9   P_MEDICAL       525 non-null    float64       
 10  QUANTITY        525 non-null    float64       
 11  QUANTITY_MA     525 non-null    float64       
 12  SEJ_HAD         525 non-null    float64       
 13  SEJ_MCO         525 non-null    float64       
 14  SEJ_PSY         525 non-null    float64       
 15  SEJ_SL

In [51]:
df_h4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CODE_ATC        525 non-null    int64         
 1   DATE            525 non-null    datetime64[ns]
 2   HOSPI_CODE_UCD  525 non-null    int64         
 3   LIT_HC          525 non-null    float64       
 4   LIT_HP          525 non-null    float64       
 5   MONTH           525 non-null    int64         
 6   N_UFS           525 non-null    float64       
 7   PN_MEDICAL      525 non-null    float64       
 8   POPULATION      525 non-null    float64       
 9   P_MEDICAL       525 non-null    float64       
 10  QUANTITY        525 non-null    float64       
 11  QUANTITY_MA     525 non-null    float64       
 12  SEJ_MCO         525 non-null    float64       
 13  SEJ_PSY         525 non-null    float64       
 14  SEJ_SLD         525 non-null    float64       
 15  SEJ_SS

## 6.1 - New database composition based on clusters

In [58]:
df_h1_cluster_0 = df_h1[df_h1['CLUSTER'] == 0].copy()
df_h1_cluster_1 = df_h1[df_h1['CLUSTER'] == 1].copy()
df_h1_cluster_2 = df_h1[df_h1['CLUSTER'] == 2].copy()
df_h1_cluster_3 = df_h1[df_h1['CLUSTER'] == 3].copy()

In [59]:
df_h2_cluster_0 = df_h2[df_h2['CLUSTER'] == 0].copy()
df_h2_cluster_1 = df_h2[df_h2['CLUSTER'] == 1].copy()
df_h2_cluster_2 = df_h2[df_h2['CLUSTER'] == 2].copy()
df_h2_cluster_3 = df_h2[df_h2['CLUSTER'] == 3].copy()

In [60]:
df_h3_cluster_0 = df_h3[df_h3['CLUSTER'] == 0].copy()
df_h3_cluster_1 = df_h3[df_h3['CLUSTER'] == 1].copy()
df_h3_cluster_2 = df_h3[df_h3['CLUSTER'] == 2].copy()
df_h3_cluster_3 = df_h3[df_h3['CLUSTER'] == 3].copy()

In [61]:
df_h4_cluster_0 = df_h4[df_h4['CLUSTER'] == 0].copy()
df_h4_cluster_1 = df_h4[df_h4['CLUSTER'] == 1].copy()
df_h4_cluster_2 = df_h4[df_h4['CLUSTER'] == 2].copy()
df_h4_cluster_3 = df_h4[df_h4['CLUSTER'] == 3].copy()

## 6.2 - Baseline score, Test 1

In [52]:
def test_1_baseline(df, medicine, df_scores, hospital = '-', unified = False):

  df = df.fillna(0)
  scaler = StandardScaler()

  X = df[df['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY', 'DATE', 'WEEK', 'CLUSTER'], axis=1).copy().values
  X_scaled = scaler.fit_transform(X)

  y = df[df['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].copy().values

  if unified:
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.35, random_state=42, shuffle = True)

  else:
    X_train = X_scaled
    X_test = X_scaled
    y_train = y
    y_test = y

  # Define the parameter distributions for RandomizedSearchCV
  param_distributions = {
      'max_depth': np.arange(2, 31, 2),
      'n_estimators': np.arange(2, 201, 2),
      'max_features': ['sqrt', 'log2'],
      'min_samples_split': np.arange(2, 11, 2),
      'min_samples_leaf': np.arange(2, 5, 1)
  }

  # Create the RandomizedSearchCV object
  randomized_search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42),
                                          param_distributions=param_distributions,
                                          n_iter=100,
                                          cv=5,
                                          random_state=42)

  # Fit the RandomizedSearchCV object to the data
  randomized_search.fit(X_train, y_train)

  # Get the best estimator
  best_estimator = randomized_search.best_estimator_

  # Make predictions using the best estimator
  y_pred = best_estimator.predict(X_test)

  # Calculate R^2 score
  r2 = r2_score(y_test, y_pred)

  # Calculate MAE
  mae = mean_absolute_error(y_test, y_pred)

  # Calculate MAPE
  mape = mean_absolute_percentage_error(y_test, y_pred)

  # Calculate RMSE
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Print the best parameters, best score, and evaluation metrics
  print('Medicine:' + str(medicine))
  print('Best Parameters:', randomized_search.best_params_)
  print('Best Score:', randomized_search.best_score_)
  print('R^2 Score:', round(r2, 3))
  print('MAE:', round(mae, 3))
  print('MAPE:', round(mape, 3))
  print('RMSE:', round(rmse, 3))
  print()


  # Create the new row as a DataFrame
  new_row = pd.DataFrame({'ID_SITE_RATTACHE': [hospital],
                          'HOSPI_CODE_UCD': ['CODE_UCD_'+str(medicine)],
                          'R2': [r2],
                          'RMSE': [rmse],
                          'MAE': [mae],
                          'MAPE': [mape]})

  # Append the new row to the DataFrame
  df_scores = pd.concat([df_scores, new_row], ignore_index=True)

  # Return the updated DataFrame
  return df_scores


In [16]:
df_prediction_scores_h1 = pd.DataFrame(columns=['ID_SITE_RATTACHE', 'HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in df_h1.HOSPI_CODE_UCD.unique():

  df_prediction_scores_h1 = test_1_baseline(df_h1, medicine, df_prediction_scores_h1, hospital = 'HOSPI_1', unified = False)

df_prediction_scores_h1

df_prediction_scores_h1.to_csv('df_prediction_scores_h1.csv')

Medicine:3400890837149
Best Parameters: {'n_estimators': 56, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 8}
Best Score: -0.9345859727694268
R^2 Score: 0.551
MAE: 96.027
MAPE: 0.035
RMSE: 144.669

Medicine:3400893826706
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 28}
Best Score: -0.10578437610201927
R^2 Score: 0.469
MAE: 268.737
MAPE: 0.094
RMSE: 382.393

Medicine:3400892729589
Best Parameters: {'n_estimators': 34, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 8}
Best Score: -0.10257021896869647
R^2 Score: 0.385
MAE: 417.36
MAPE: 0.13
RMSE: 575.272

Medicine:3400892088310
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 28}
Best Score: -20.30800255519136
R^2 Score: 0.486
MAE: 188.153
MAPE: 0.077
RMSE: 241.974

Medicine:3400892075761
Best Parameters: {'n_estima

In [17]:
df_prediction_scores_h2 = pd.DataFrame(columns=['ID_SITE_RATTACHE', 'HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in df_h2.HOSPI_CODE_UCD.unique():

  df_prediction_scores_h2 = test_1_baseline(df_h2, medicine, df_prediction_scores_h2, hospital = 'HOSPI_2', unified = False)

df_prediction_scores_h2

df_prediction_scores_h2.to_csv('df_prediction_scores_h2.csv')

Medicine:3400890837149
Best Parameters: {'n_estimators': 148, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 30}
Best Score: -0.14626461366990948
R^2 Score: 0.627
MAE: 188.889
MAPE: 0.075
RMSE: 235.885

Medicine:3400893826706
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 28}
Best Score: -0.26257315570552675
R^2 Score: 0.657
MAE: 139.93
MAPE: 0.078
RMSE: 185.747

Medicine:3400892729589
Best Parameters: {'n_estimators': 56, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 8}
Best Score: -0.2481624117726819
R^2 Score: 0.719
MAE: 263.218
MAPE: 0.057
RMSE: 318.477

Medicine:3400892088310
Best Parameters: {'n_estimators': 172, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10}
Best Score: 0.07933661691957819
R^2 Score: 0.626
MAE: 175.503
MAPE: 0.063
RMSE: 243.87

Medicine:3400892075761
Best Parameters: {'n_es

In [18]:
df_prediction_scores_h3 = pd.DataFrame(columns=['ID_SITE_RATTACHE', 'HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in df_h3.HOSPI_CODE_UCD.unique():

  df_prediction_scores_h3 = test_1_baseline(df_h3, medicine, df_prediction_scores_h3, hospital = 'HOSPI_3', unified = False)

df_prediction_scores_h3

df_prediction_scores_h3.to_csv('df_prediction_scores_h3.csv')

Medicine:3400890837149
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 28}
Best Score: 0.16132226619237966
R^2 Score: 0.608
MAE: 188.127
MAPE: 0.047
RMSE: 229.33

Medicine:3400893826706
Best Parameters: {'n_estimators': 2, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 18}
Best Score: -0.4279407699851888
R^2 Score: 0.386
MAE: 703.1
MAPE: 0.095
RMSE: 819.365

Medicine:3400892729589
Best Parameters: {'n_estimators': 104, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 22}
Best Score: 0.18657904047631652
R^2 Score: 0.597
MAE: 264.207
MAPE: 0.04
RMSE: 331.898

Medicine:3400892088310
Best Parameters: {'n_estimators': 108, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 8}
Best Score: -0.23768492471961644
R^2 Score: 0.315
MAE: 245.162
MAPE: 0.059
RMSE: 334.992

Medicine:3400892075761
Best Parameters: {'n_estima

In [19]:
df_prediction_scores_h4 = pd.DataFrame(columns=['ID_SITE_RATTACHE', 'HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in df_h4.HOSPI_CODE_UCD.unique():

  df_prediction_scores_h4 = test_1_baseline(df_h4, medicine, df_prediction_scores_h4, hospital = 'HOSPI_4', unified = False)

df_prediction_scores_h4

df_prediction_scores_h4.to_csv('df_prediction_scores_h4.csv')

Medicine:3400890837149
Best Parameters: {'n_estimators': 36, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10}
Best Score: -0.23256647837501715
R^2 Score: 0.151
MAE: 1430.946
MAPE: 0.087
RMSE: 1848.54

Medicine:3400893826706
Best Parameters: {'n_estimators': 56, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 8}
Best Score: 0.16664580849231403
R^2 Score: 0.528
MAE: 632.968
MAPE: 0.123
RMSE: 786.121

Medicine:3400892729589
Best Parameters: {'n_estimators': 4, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 30}
Best Score: -0.3703202361499293
R^2 Score: 0.069
MAE: 2060.769
MAPE: 0.133
RMSE: 2747.893

Medicine:3400892088310
Best Parameters: {'n_estimators': 4, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 30}
Best Score: -0.25484120415025896
R^2 Score: 0.164
MAE: 1338.596
MAPE: 0.274
RMSE: 1755.035

Medicine:3400892075761
Best Parameters: {'n

In [53]:
df_prediction_scores_unified = pd.DataFrame(columns=['ID_SITE_RATTACHE', 'HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_unified = pd.concat([df_h1,df_h2,df_h3, df_h4])

for medicine in df_unified.HOSPI_CODE_UCD.unique():

  df_prediction_scores_unified = test_1_baseline(df_unified, medicine, df_prediction_scores_unified, hospital = '-', unified = True)

df_prediction_scores_unified.to_csv('df_prediction_scores_unified.csv')

Medicine:3400890837149
Best Parameters: {'n_estimators': 150, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 30}
Best Score: 0.9679595902171432
R^2 Score: 0.974
MAE: 566.924
MAPE: 0.085
RMSE: 983.941

Medicine:3400893826706
Best Parameters: {'n_estimators': 34, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 8}
Best Score: 0.8546861200438457
R^2 Score: 0.902
MAE: 553.042
MAPE: 0.136
RMSE: 765.646

Medicine:3400892729589
Best Parameters: {'n_estimators': 36, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10}
Best Score: 0.9424415434671003
R^2 Score: 0.926
MAE: 944.625
MAPE: 0.147
RMSE: 1543.928

Medicine:3400892088310
Best Parameters: {'n_estimators': 104, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 18}
Best Score: 0.49819016877091976
R^2 Score: 0.886
MAE: 427.05
MAPE: 0.107
RMSE: 574.171

Medicine:3400892075761
Best Parameters: {'n_esti

## 6.3 - Clustering score, Test 2

### Hospital 1 - Cluster 0